In [0]:
#1
#Don't have to run this for different queries just run it once every time when you open the link

#DON'T CHANGE THIS

# setting the enviroment

!sudo apt-get update
!sudo apt-get install build-essential chrpath libssl-dev libxft-dev
!sudo apt-get install libfreetype6 libfreetype6-dev libfontconfig1 libfontconfig1-dev
!wget https://bitbucket.org/ariya/phantomjs/downloads/phantomjs-2.1.1-linux-x86_64.tar.bz2
!tar xvjf phantomjs-2.1.1-linux-x86_64.tar.bz2 -C /usr/local/share/
!sudo ln -sf /usr/local/share/phantomjs-2.1.1-linux-x86_64/bin/phantomjs /usr/local/bin
!phantomjs --version
!pip install requests
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin
# !pip install selenium


In [0]:
#2
#run this everytime you change the variables and then run the #3
#setting the variables
# set all the variables 
query="cannabis+dispensaries+oakland" #set the query to search for NO SPACES allow use + instead of spaces
pagesupto = 50
keepScraping= True #change it to True if you don't want to answer yes/no each time 
domainList={'com','net','org'} #accept the emails ending with these
addressOfContactPage = {'/pages/contact','/contact-us/'} # for an example website www.google.com the code will check 1. www.google.com/pages/contact 2. www.google.com/contact-us/ 3. www.google.com    to find email
csvFileName='sample.csv' #name of file to save 


In [0]:
#3
#Don't change this
#run it after setting the variables 
from selenium import webdriver
import re
import requests
import pandas
from tqdm import tqdm_notebook as tqdmn
from tqdm import tqdm
from google.colab import files
pagesupto-=1

browser = webdriver.PhantomJS(desired_capabilities={'browserName': 'phantomjs', 'version': '', 'platform': 'ANY', 'javascriptEnabled': False}) # or add to your PATH

finalans=[]
browser.get('https://www.google.com/search?q=hi')

# Defining all the functions

def parse(place):
    toret = {}
    toret['link'] = place.get_attribute('href')
    toret['name'] = place.find_element_by_class_name('BNeawe').text
    toret['ratings'] = place.find_element_by_class_name('oqSTJd').text
    spanElm = place.find_elements_by_css_selector('span')
    for span in spanElm:
        if span.get_attribute('class') =='':
            toret['rated by'] = span.text.strip('(').strip(')').replace(',','')
    toret['rated by'] = toret['rated by']
    toret['extra data'] = place.text.split('\n')[3:]
    return toret

def filterEmail(lst):
    filst=[]
    for email in lst:
        if email.split('.')[-1] in domainList:
            filst.append(email)
    return filst
def findEmail(source):
    lst = set(re.findall('[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+', source))
    
    return filterEmail(lst)
def parseplaces(places):
    toret = []
    for place in places:
        toret.append(parse(place))
    return toret
def parseCurrPage():
    places = browser.find_elements_by_class_name('tHmfQe')
    return parseplaces(places)
def parseQuery(query):
    global finalans
    toret = []
    scraped=0
    print('searching query')
    for i in range(2):
        browser.get(f'https://www.google.com/search?safe=active&ei=MS00XcDKG9HTz7sP6vqEmAc&q={query}&rlst=f&sa=X&ved=2ahUKEwiAp9-01sXjAhXR6XMBHWo9AXMQjGowAHoECAsQDA')
    while True:
        
        info = parseCurrPage()
        toret+=info
        finalans+=info
        
        nextpage =  [elm for elm in browser.find_elements_by_class_name('qFvlD') if elm.text=='>']
        
        if browser.find_element_by_class_name('nBDE1b').text == 'Next >':
            nextpage += browser.find_elements_by_class_name('nBDE1b')
        
        if len(nextpage) ==1:
            
            
            print(scraped+1)
            if scraped>=pagesupto:
                print('pages search completed')
                break
            print('going to next page')
            nextpage[0].click()
            
        elif len(nextpage)==0:
            print('no more pages')
            break
        else:
            print('Contact me. There is something wrong')
        scraped+=1
    print('getting emails')
    for ans in tqdmn(range(len(toret))):
#         print(ans)
#         print(toret[ans]['link'])
        temps= processPlaceSearchemail(toret[ans]['link'])
        toret[ans]['email'],toret[ans]['website'] = temps['email'],temps['website']
#         print('next one')
    print('done')
    return toret
def processPlaceSearchemail(link):
    toret = {}
#     print('getting page')

    browser.get(link)
#     print('got it')
    emails = []
    i=1
#     print('selecting element')
    for something in range(10):
        try:
            divElm = browser.find_elements_by_css_selector('div.ZINbbc.xpd.uUPGi')[i]
            linkElm = divElm.find_element_by_css_selector('div.skVgpb')
            
            break
        except:
            i+=1
    try:
        weblink = linkElm.find_elements_by_css_selector('a.VGHMXd')[-1]
    except:
        toret['email'] = None
        toret['website'] = None
        return toret
    rawlink = weblink.get_attribute('href')
    processedlink = rawlink[rawlink.find('http',4):rawlink.find('&sa=U')]
    toret['website'] = processedlink
    
#     print('going to website')
    try:
        emails= findEmail(requests.get(processedlink+'/pages/contact').text)
    except:
        emails = []
#     print('checking further options')
    for address in addressOfContactPage:
        try:
            emails+=findEmail(requests.get(processedlink+address).text)
        except:
            pass
#     print('found emails')
    emails = set(emails)
    
    if(len(emails)!=0):
        toret['email'] = emails
    else:
        toret['email'] = None
    return toret
#making the search

finalquery = parseQuery(query)
#processing data to save to file
print('query completed, now processing to make file')

j = [list(f.values()) for f in finalquery]
j = [f[1:]+f[:1] for f in j]
print('creating pandas dataframe')
csv = pandas.DataFrame(j,columns=['name','ratings','rated by','extra info','email','website','link'])
print('file ready to save')
#save the file 
csv.to_csv(csvFileName)
print('you can download the "sample.csv" file by going to the little arrow in top left corner just down to "+Code" button and then to files options.')
print("contents of file:-")
csv